In [11]:
import cv2
import dlib
import numpy as np

In [12]:
# Load the PNG image of the clown nose
clown_nose = cv2.imread("clown_nose.png", cv2.IMREAD_UNCHANGED)
blush = cv2.imread("blush.png", cv2.IMREAD_UNCHANGED)

# Initialize video capture and face detector
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [13]:
# Variables to toggle features
show_landmarks = False
show_clown_nose = False
show_blush = False

In [14]:
def draw_landmarks(frame, landmarks):
    # Draw circles at each facial landmark point
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        cv2.circle(frame, (x, y), 3, (255, 0, 0), -1)

In [15]:
def apply_clown_nose(frame, landmarks):
    # Extract the coordinates of the nose tip (landmark 33)
    nose_tip = (landmarks.part(33).x, landmarks.part(33).y)
    
    # Resize the clown nose to fit the size of the nose
    width = int(landmarks.part(16).x - landmarks.part(0).x) // 3
    height = int(clown_nose.shape[0] * (width / clown_nose.shape[1]))
    resized_nose = cv2.resize(clown_nose, (width, height))
    
    # Calculate the position to place the clown nose
    x_offset = nose_tip[0] - width // 2
    y_offset = nose_tip[1] - height // 2 - 10  # Adjust height up
    
    # Create a region of interest (ROI) in the frame
    roi = frame[y_offset:y_offset + height, x_offset:x_offset + width]
    
    # Add the clown nose image to the ROI using alpha blending
    alpha_s = resized_nose[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    
    for c in range(0, 3):
        roi[:, :, c] = (alpha_s * resized_nose[:, :, c] + alpha_l * roi[:, :, c])
    
    # Place the modified ROI back into the frame
    frame[y_offset:y_offset + height, x_offset:x_offset + width] = roi

In [16]:
def apply_blush(frame, landmarks):
    # Calculate the size and position for the blush overlay
    width = int((landmarks.part(16).x - landmarks.part(0).x) / 3)
    height = int(blush.shape[0] * (width / blush.shape[1]))
    resized_blush = cv2.resize(blush, (width, height))
    
    # Calculate the positions for the blush on both cheeks
    left_cheek = (landmarks.part(3).x - width // 2, landmarks.part(3).y - height // 1)
    right_cheek = (landmarks.part(13).x - width // 2, landmarks.part(13).y - height // 1)
    
    # Function to apply the blush on a cheek
    def overlay_blush(cheek_position):
        x, y = cheek_position
        # Create a region of interest (ROI)
        roi = frame[y:y + height, x:x + width]
        
        # Add the blush image to the ROI using alpha blending
        alpha_s = resized_blush[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        
        for c in range(0, 3):
            roi[:, :, c] = (alpha_s * resized_blush[:, :, c] + alpha_l * roi[:, :, c])
        
        # Place the modified ROI back into the frame
        frame[y:y + height, x:x + width] = roi
    
    # Apply the blush to both cheeks
    overlay_blush(left_cheek)
    overlay_blush(right_cheek)

In [17]:
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = detector(gray)
    for face in faces:
        # Get facial landmarks
        landmarks = predictor(gray, face)
        
        # Toggle features based on flags
        if show_landmarks:
            draw_landmarks(frame, landmarks)
        
        if show_clown_nose:
            apply_clown_nose(frame, landmarks)
            
        if show_blush:
            apply_blush(frame, landmarks)
    
    # Display the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)
    
    # Handle keyboard input to toggle features and exit
    if key == ord('l'):  # Press 'l' to toggle landmarks
        show_landmarks = not show_landmarks
    elif key == ord('c'):  # Press 'c' to toggle clown nose
        show_clown_nose = not show_clown_nose
    elif key == ord('b'):  # Press 'b' to toggle blush filter
        show_blush = not show_blush
    elif key == 27:  # Press 'Esc' to exit
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()
